In [ ]:
%pip install pmdarima

In [1]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
import os
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

print("🚀 ARIMA TOMORROW PREDICTION - Nifty 200 (1-Year Data) - FIXED")

🚀 ARIMA TOMORROW PREDICTION - Nifty 200 (1-Year Data) - FIXED


In [3]:
# Load data
df = pd.read_csv("nifty200_data/nifty200_complete.csv")
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)

In [4]:
# 1-Year data
one_year_ago = df.index[-252] if len(df) >= 252 else df.index[0]
df_1year = df.loc[one_year_ago:]
print(f"📊 1-Year data: {len(df_1year)} days")

# ✅ FIXED: Get column names properly
open_cols = []
for col in df_1year.columns:
    if str(col).endswith('_Open') and df_1year[col].notna().sum() > 100:
        open_cols.append(col)

print(f"🎯 Valid stocks: {len(open_cols)}")

forecast_dir = "nifty200_arima_fixed"
os.makedirs(forecast_dir, exist_ok=True)

tomorrow_date = df_1year.index[-1] + timedelta(days=1)

📊 1-Year data: 252 days
🎯 Valid stocks: 200


In [5]:
# ========================================
# ARIMA PREDICTION FUNCTION - ROBUST
# ========================================
def safe_arima_predict(stock_series):
    """Safe ARIMA prediction with error handling"""
    try:
        # Quick ARIMA fit
        model = auto_arima(
            stock_series.dropna().values,
            start_p=0, start_q=0, max_p=2, max_q=2,
            d=1,           # Price differencing
            seasonal=False,
            stepwise=True,
            suppress_warnings=True,
            maxiter=30,
            alpha=0.05,
            n_fits=10      # Fast fitting
        )
        
        # Predict tomorrow
        forecast = model.predict(n_periods=1)
        return forecast[0], model.order, model.aic()
    
    except:
        return None, None, None

In [6]:
# ========================================
# MAIN LOOP WITH ROBUST ERROR HANDLING
# ========================================
print("\n🔮 Running ARIMA predictions...")
results = []

for i, stock_col in enumerate(open_cols):
    print(f"[{i+1:3d}/{len(open_cols)}] {stock_col}", end=" ")
    
    stock_data = df_1year[stock_col]
    
    tomorrow_price, arima_order, aic = safe_arima_predict(stock_data)
    
    if tomorrow_price is not None and not np.isnan(tomorrow_price):
        last_price = stock_data.dropna().iloc[-1]
        
        if last_price > 0:  # Valid price
            change_pct = ((tomorrow_price / last_price) - 1) * 100
            
            results.append({
                'Stock': str(stock_col).replace('Open_', ''),
                'Last_Close': round(float(last_price), 2),
                'Tomorrow_Open_ARIMA': round(float(tomorrow_price), 2),
                'Change_Pct': round(float(change_pct), 2),
                'ARIMA_Order': f"({arima_order[0]},{arima_order[1]},{arima_order[2]})" if arima_order else "N/A",
                'AIC': round(float(aic), 1) if aic else 0.0,
                'Signal': '🟢 BUY' if change_pct > 1.0 else '🟡 HOLD' if change_pct > -1.0 else '🔴 SELL'
            })
            print(f"✅ {change_pct:+.2f}%")
        else:
            print("❌ Invalid price")
    else:
        print("❌ ARIMA failed")
    
    if (i + 1) % 50 == 0:
        print(f"Progress: {i+1}/{len(open_cols)}")


🔮 Running ARIMA predictions...
[  1/200] 360ONE.NS_Open ✅ +0.00%
[  2/200] ABB.NS_Open ✅ +0.08%
[  3/200] ABCAPITAL.NS_Open ✅ +0.28%
[  4/200] ACC.NS_Open ✅ +0.28%
[  5/200] ADANIENSOL.NS_Open ✅ -0.12%
[  6/200] ADANIENT.NS_Open ✅ +0.12%
[  7/200] ADANIGREEN.NS_Open ✅ +0.20%
[  8/200] ADANIPORTS.NS_Open ✅ +0.07%
[  9/200] ADANIPOWER.NS_Open ✅ +0.17%
[ 10/200] ALKEM.NS_Open ✅ +0.00%
[ 11/200] AMBUJACEM.NS_Open ✅ -0.03%
[ 12/200] APLAPOLLO.NS_Open ✅ +0.14%
[ 13/200] APOLLOHOSP.NS_Open ✅ +0.00%
[ 14/200] ASHOKLEY.NS_Open ✅ +0.18%
[ 15/200] ASIANPAINT.NS_Open ✅ +0.00%
[ 16/200] ASTRAL.NS_Open ✅ +0.00%
[ 17/200] ATGL.NS_Open ✅ +0.50%
[ 18/200] AUBANK.NS_Open ✅ +0.24%
[ 19/200] AUROPHARMA.NS_Open ✅ +0.33%
[ 20/200] AXISBANK.NS_Open ✅ +0.00%
[ 21/200] BAJAJ-AUTO.NS_Open ✅ +0.00%
[ 22/200] BAJAJFINSV.NS_Open ✅ +0.01%
[ 23/200] BAJAJHFL.NS_Open ✅ +0.22%
[ 24/200] BAJAJHLDNG.NS_Open ✅ -0.21%
[ 25/200] BAJFINANCE.NS_Open ✅ +0.00%
[ 26/200] BANKBARODA.NS_Open ✅ +0.17%
[ 27/200] BANKINDIA.NS_Open 

In [7]:
# ========================================
# ✅ FIXED SAVE SECTION
# ========================================
print(f"\n📊 Results: {len(results)} successful predictions")

if len(results) > 0:
    results_df = pd.DataFrame(results)
    results_df['Forecast_Date'] = tomorrow_date.strftime('%Y-%m-%d')
    
    # MAIN FILES
    results_df.to_csv(f"{forecast_dir}/ARIMA_TOMORROW_NIFTY200.csv", index=False)
    
    # ✅ SAFE TOP MOVERS (check column exists)
    if 'Change_Pct' in results_df.columns:
        results_df.nlargest(15, 'Change_Pct').to_csv(f"{forecast_dir}/ARIMA_TOP15_GAINERS.csv", index=False)
        results_df.nsmallest(15, 'Change_Pct').to_csv(f"{forecast_dir}/ARIMA_TOP15_LOSERS.csv", index=False)
        
        # Display results
        print(f"\n🎉 SUCCESS! Files saved to: {forecast_dir}/")
        print(f"📁 ARIMA_TOMORROW_NIFTY200.csv ({len(results_df)} stocks)")
        
        print("\n🔥 TOP 10 GAINERS:")
        print(results_df.nlargest(10, 'Change_Pct')[['Stock', 'Change_Pct', 'Signal']].to_string(index=False))
        
        print("\n📉 TOP 10 LOSERS:")
        print(results_df.nsmallest(10, 'Change_Pct')[['Stock', 'Change_Pct', 'Signal']].to_string(index=False))
        
        # Signal counts
        signals = results_df['Signal'].value_counts()
        print(f"\n📈 SIGNALS:")
        for signal, count in signals.items():
            print(f"  {signal}: {count}")
    
else:
    print("❌ No successful predictions. Check data format.")

print(f"\n📅 Tomorrow: {tomorrow_date.strftime('%Y-%m-%d')}")


📊 Results: 200 successful predictions

🎉 SUCCESS! Files saved to: nifty200_arima_fixed/
📁 ARIMA_TOMORROW_NIFTY200.csv (200 stocks)

🔥 TOP 10 GAINERS:
             Stock  Change_Pct Signal
   YESBANK.NS_Open        1.65  🟢 BUY
     CIPLA.NS_Open        0.86 🟡 HOLD
FEDERALBNK.NS_Open        0.81 🟡 HOLD
       PNB.NS_Open        0.79 🟡 HOLD
ULTRACEMCO.NS_Open        0.75 🟡 HOLD
  SONACOMS.NS_Open        0.69 🟡 HOLD
BHARATFORG.NS_Open        0.56 🟡 HOLD
      ATGL.NS_Open        0.50 🟡 HOLD
TORNTPOWER.NS_Open        0.50 🟡 HOLD
BLUESTARCO.NS_Open        0.47 🟡 HOLD

📉 TOP 10 LOSERS:
             Stock  Change_Pct Signal
       LTF.NS_Open       -0.83 🟡 HOLD
    RECLTD.NS_Open       -0.63 🟡 HOLD
 BANKINDIA.NS_Open       -0.38 🟡 HOLD
 POLICYBZR.NS_Open       -0.37 🟡 HOLD
JINDALSTEL.NS_Open       -0.32 🟡 HOLD
 SUNPHARMA.NS_Open       -0.29 🟡 HOLD
   CGPOWER.NS_Open       -0.25 🟡 HOLD
       HAL.NS_Open       -0.25 🟡 HOLD
GMRAIRPORT.NS_Open       -0.24 🟡 HOLD
     LUPIN.NS_Open       -0.24 🟡 